# Langfuse Masking Issue MVP

In [1]:
import re
import os
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI
from langfuse import Langfuse
from langfuse.langchain import CallbackHandler

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""
os.environ["LANGFUSE_HOST"] = ""

In [ ]:
def current_masking_function(data: Any, **kwargs) -> Any:
    if isinstance(data, list):
        data = [current_masking_function(item, **kwargs) for item in data]
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(key, str):
                data[key] = "*****" if key.startswith("secret") else data[key]
            if isinstance(value, str):
                data[key] = re.sub(
                    r"(<secret_start>)(.*?)(<secret_end>)", 
                    r"\1*****\3", 
                    data[key], 
                    flags=re.DOTALL
                )
    return data


def create_tracing_config(tags: str | list[str]) -> RunnableConfig:
    if isinstance(tags, str):
        tags = [tags]

    return RunnableConfig(
        callbacks=[CallbackHandler()],
        metadata={
            "langfuse_tags": tags,
        },
    )


langfuse_client = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST"),
    mask=current_masking_function,
)

In [ ]:
prompt_template = """
What is the value of the secret?
<secret_start>
{secret}
<secret_end>
"""


prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(model="gpt-4.1", temperature=0)
chain = prompt | model


secret = "FIFA12345"
response = chain.invoke({"secret": secret}, config=create_tracing_config("test_masking"))


print(f"Response: {response.content}")

Response: The value of the secret is:

FIFA12345
